 # Opis notatnika

 Ten notatnik inicjuje naszą pracę nad warsztatem końcowym. Naszym zadaniem tutaj jest pobranie udostępnionych nam danych do obszaru roboczego, które w następnym kroku wgramy na naszą bazę danych. Ich obróbka oraz analiza zostanie przeprowadzona w specjalnie do tego celu przygotowanych kolejnych notatnikach.

 Na potrzeby tego warsztatu został stworzony dedykowany serwis API, który dostępny jest pod adresem: https://api-datalab.coderslab.com/api/. Dodatkowo udostępniona została dokumentacja, z którą można zapoznać się tutaj: [klik](https://api-datalab.coderslab.com/docs/).

 > Dokumentacja jest czysto techniczna i ma na celu prezentację dostępnych endpointów wraz ze zwracanym typem. W celu przetestowania należy kliknąć przysisk `Authorize`, podać token (dostępny poniżej), a następnie `Try it out!` oraz uzupełnić wymagane pola (parametry requesta).

 Zgodnie z dokumentacją stwierdzamy, że udostępnione zostały nam 4 endpointy:
 - `airport` - dane o lotnisku,
 - `weather` - informacje o zarejestrowaniej pogodzie na lotnisku danego dnia,
 - `aircraft` - dane o samolotach
 - `flights` - dane o wylotach z danego lotniska per dzień.

 Wszystkie te źródła musimy pobrać, aby być w stanie wykonać całość warsztatu. W celu pobrania informacji, gdzie wymagany jest paramatr `airportId`, posłużymy się listą z pliku `airports.csv`.

 Przy wykonywaniu tego zadania możesz posłużyć się tym tokenem: `WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr`.

 ### Uwagi
 - Ze względów ćwiczeniowych, konstrukcja poszczególnych endpointów jest różna – w trakcie pracy dokładnie przyjrzyj się, w jaki sposób należy wykonać zapytanie, aby otrzymać odpowiedź.
 - Pamiętaj o dodaniu `sleep` pomiędzy poszczególnymi wywołaniami endpoint.
 - Limit wywołań API to 500/min, zadbaj o nieprzekroczenie tego limitu – w przeciwnym wypadku będzie zwracany błąd 429.

 # Konfiguracja notatnika

 Tutaj zaimportuj wymagane biblioteki

In [1]:
import requests
import json
from time import sleep
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

 Tutaj zdefiniuj parametry połączenia do API

In [2]:
api_key = "WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr"
authorization = {'authorization': api_key}

Tutaj wczytaj plik `airports.csv` i dostosuj do dalszych kroków w celu pobierania z kolejnych endpointów. Lista lotnisk jest dostępna w kolumnie `origin_airport_id`.

In [4]:
airports = pd.read_csv("/Users/Me/desktop/Airports_CL/data/airports.csv")
airports.head()

,origin_airport_id
0,10874
1,11233
2,13360
3,15008
4,11638


#zamiana pliku csv w listę

In [5]:
airports = pd.read_csv("/Users/Me/desktop/Airports_CL/data/airports.csv", header = 0)
airports_list = list(airports.origin_airport_id)

#stworzenie pętli na podstawie 'airports_list' w cely ustalenia, które lotniska zwracają status połączenia = 200. Następnie stworzenie nowej listy 'true_airports', zawierającej tylko lotniska zwracające status połączenia = 200. 

In [6]:
url_airport = "https://api-datalab.coderslab.com/api/airport/{airportID}"
airports_list = list(airports.origin_airport_id)
true_airports = []

for airportID in airports_list:
    url = url_airport.replace("{airportID}", str(airportID))
    response = requests.get(url, headers = authorization)
    
    if response.status_code == 200:
        true_airports.append(airportID)

#stworzenie dictionary "airport_df"

In [7]:
airport_df = {}

for airportID in true_airports:
    url = url_airport.replace("{airportID}", str(airportID))
    response = requests.get(url, headers = authorization)
    
    if response.status_code == 200:
        airport_data = response.json()
        airport_df[airportID] = airport_data
    
print(airport_df)

{11638: {'ORIGIN_AIRPORT_ID': 11638, 'DISPLAY_AIRPORT_NAME': 'Fresno Air Terminal', 'ORIGIN_CITY_NAME': 'Fresno, CA', 'NAME': 'FRESNO YOSEMITE INTERNATIONAL, CA US'}, 13342: {'ORIGIN_AIRPORT_ID': 13342, 'DISPLAY_AIRPORT_NAME': 'General Mitchell Field', 'ORIGIN_CITY_NAME': 'Milwaukee, WI', 'NAME': 'MILWAUKEE MITCHELL AIRPORT, WI US'}, 13244: {'ORIGIN_AIRPORT_ID': 13244, 'DISPLAY_AIRPORT_NAME': 'Memphis International', 'ORIGIN_CITY_NAME': 'Memphis, TN', 'NAME': 'MEMPHIS INTERNATIONAL AIRPORT, TN US'}, 15096: {'ORIGIN_AIRPORT_ID': 15096, 'DISPLAY_AIRPORT_NAME': 'Syracuse Hancock International', 'ORIGIN_CITY_NAME': 'Syracuse, NY', 'NAME': 'SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US'}, 10397: {'ORIGIN_AIRPORT_ID': 10397, 'DISPLAY_AIRPORT_NAME': 'Atlanta Municipal', 'ORIGIN_CITY_NAME': 'Atlanta, GA', 'NAME': 'ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US'}, 10529: {'ORIGIN_AIRPORT_ID': 10529, 'DISPLAY_AIRPORT_NAME': 'Bradley International', 'ORIGIN_CITY_NAME': 'Hartford, CT', 'N

 # Pobieranie `Airport`
 Zapoznaj się z dokumentacją endpointu `airport`, a następnie pobierz dane dot. poszczególnych lotnisk. Wyniki tego kroku zapisz do ramki `airport_df`, a następnie zapisz do pliku `csv`.

 ### Wskazówki
 - Nie wszystkie lotniska dostępne w pliku `airports.csv`, są dostępne w endpoint. Zadbaj o odpowiednie obsłużenie takiej sytuacji,
 - Do skonwertowania wyników przydatna może okazać się metoda `Pandas` - [from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html),
 - Artykuł LMS: `Python - analiza danych > Dzień 4 - API > Uwierzytelnianie`
 - Artykuł LMS: `Python - analiza danych > Przygotowanie do zjazdu 2`

 Tutaj pobierz dane z endpoint'u `airport`

#sprawdzenie połączenia

In [8]:
url_airport = "https://api-datalab.coderslab.com/api/airport/{airportID}"
response = requests.get(url_airport, headers = authorization)
response.status_code

400

#stworzenie DataFrame "airport_df" na podstawie dictionary "airport_df"

In [9]:
airport_df = pd.DataFrame.from_records(airport_df)
airport_df = airport_df.transpose() #odwrócenie tabeli
airport_df

,ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
10140,10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"
10257,10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"
10299,10299,Anchorage International,"Anchorage, AK","ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, A..."
10397,10397,Atlanta Municipal,"Atlanta, GA",ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...
10423,10423,Austin - Bergstrom International,"Austin, TX","AUSTIN BERGSTROM INTERNATIONAL AIRPORT, TX US"
...,...,...,...,...
15304,15304,Tampa International,"Tampa, FL","TAMPA INTERNATIONAL AIRPORT, FL US"
15370,15370,Tulsa International,"Tulsa, OK","OKLAHOMA CITY WILL ROGERS WORLD AIRPORT, OK US"
15376,15376,Tucson International,"Tucson, AZ","PHOENIX AIRPORT, AZ US"
15412,15412,McGhee Tyson,"Knoxville, TN","KNOXVILLE AIRPORT, TN US"


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [10]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

działa :)

 Tutaj zapisz ramkę `airport_df` do pliku `airport_list.csv`.

In [11]:
airport_df.to_csv("/Users/Me/desktop/Airports_CL/data/raw/airport_list.csv", index=False)

 # Pobieranie `Weather`
 Zapoznaj się z dokumentacją endpotu `Weather`, następnie pobierz dane dotyczące zarejestrowanej pogody na poszczególnych lotniskach. Wyniki zapisz do ramki `weather_df`, a później do pliku `airport_weather.csv`.

 Wskazówki:
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`, czyli stworzenie listy, a następnie przekonwertowanie jej w postać ramki.
 - Data początkowa danych to `2019-01-01`, zaś data końcowa to `2020-03-31`, czyli 15 miesięcy,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.
 - Przy dodawaniu miesięcy do daty może przydać się metoda [relativedelta](https://www.geeksforgeeks.org/python-get-month-from-year-and-weekday/).

#sprawdzenie połączenia

In [3]:
url_weather = "https://api-datalab.coderslab.com/api/airportWeather?date={value}"
response = requests.get(url_weather, headers = authorization)
response.status_code

200

#stworzenie listy dat

In [4]:
from datetime import datetime, timedelta

start_date = datetime(2019, 1, 1)
num_months = 15

dates = []

for i in range(num_months):
    dates.append(start_date.strftime("%Y-%m"))
    if start_date.month == 12:
        start_date = start_date.replace(year=start_date.year + 1, month=1)
    else:
        start_date = start_date.replace(month=start_date.month + 1)

print(dates)

['2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-02', '2020-03']


#pobieranie danych z API w formacie json / print(dates) dla sprawdzenia progresu

In [5]:
airport_weather_df = pd.DataFrame()
for date in dates:
    url = url_weather.replace("{value}", str(date))
    response = requests.get(url, headers = authorization)
    print(str(date))
    data = response.json()
    df_single_w = pd.json_normalize(data)
    airport_weather_df = airport_weather_df.append(df_single_w)

2019-01


/var/folders/5w/n4j85_qn65vg8_tq2dsjly100000gn/T/ipykernel_7841/71197393.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airport_weather_df = airport_weather_df.append(df_single_w)


2019-02


/var/folders/5w/n4j85_qn65vg8_tq2dsjly100000gn/T/ipykernel_7841/71197393.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airport_weather_df = airport_weather_df.append(df_single_w)


2019-03


...

/var/folders/5w/n4j85_qn65vg8_tq2dsjly100000gn/T/ipykernel_7841/71197393.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airport_weather_df = airport_weather_df.append(df_single_w)


2020-02


/var/folders/5w/n4j85_qn65vg8_tq2dsjly100000gn/T/ipykernel_7841/71197393.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airport_weather_df = airport_weather_df.append(df_single_w)


2020-03


/var/folders/5w/n4j85_qn65vg8_tq2dsjly100000gn/T/ipykernel_7841/71197393.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  airport_weather_df = airport_weather_df.append(df_single_w)


In [6]:
airport_weather_df

,WT18,STATION,NAME,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,...,PGTM,WT10,WESD,SN32,SX32,PSUN,TSUN,TOBS,WT07,WT11
0,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-01,4.70,0.14,0.0,0.0,64.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-02,4.92,0.57,0.0,0.0,56.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-03,5.37,0.15,0.0,0.0,52.0,55.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-04,12.08,1.44,0.0,0.0,56.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,None,USW00013874,ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPO...,2019-01-05,13.42,0.00,0.0,0.0,49.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2568,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-27,3.58,0.21,NaN,NaN,NaN,59.0,...,146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2569,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-28,6.93,1.29,NaN,NaN,NaN,77.0,...,1535.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2570,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-29,16.55,0.02,NaN,NaN,NaN,78.0,...,1408.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2571,None,USW00014762,"PITTSBURGH ALLEGHENY CO AIRPORT, PA US",2020-03-30,13.42,0.00,NaN,NaN,NaN,57.0,...,817.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [7]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

działa :)

 ## Zapis do pliku
 Tutaj zapisz ramkę `weather_df` do pliku `airport_weather.csv` w katalogu `data/raw`.

In [8]:
airport_weather_df.to_csv("/Users/Me/desktop/Airports_CL/data/raw/airport_weather.csv", index=False)

 # Pobranie `Aircraft`
 Zapoznaj się z dokumentacją endpointu `aircraft` a następnie pobierz dane produkcyjne samolotów. Wyniki zapisz do ramki `aircraft_df`, a następnie zapisz do pliku `aircraft.csv`.


#sprawdzenie połączenia

In [3]:
url_aircraft = "https://api-datalab.coderslab.com/api/Aircraft"
response = requests.get(url_aircraft, headers = authorization)
response.status_code

200

#pozyskiwanie danych i umieszczanie ich w ramce

In [5]:
aircraft_df = pd.DataFrame()
data_aircraft = response.json()
variable_for_normalization = pd.json_normalize(data_aircraft)
aircraft_df = aircraft_df.append(variable_for_normalization)
aircraft_df

/var/folders/5w/n4j85_qn65vg8_tq2dsjly100000gn/T/ipykernel_8072/2850412998.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  aircraft_df = aircraft_df.append(variable_for_normalization)


,MANUFACTURE_YEAR,TAIL_NUM,NUMBER_OF_SEATS
0,1944,N54514,0.0
1,1945,N1651M,0.0
2,1953,N100CE,0.0
3,1953,N141FL,0.0
4,1953,N151FL,0.0
...,...,...,...
7378,2019,N14011,337.0
7379,2019,N16008,337.0
7380,2019,N16009,337.0
7381,2019,N2250U,276.0


 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [6]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

#działa :)

 ## Zapis do pliku
 Tutaj zapisz ramkę `aircraft_df` do pliku `aircraft.csv` w katalogu `data/raw`.

In [7]:
aircraft_df.to_csv("/Users/Me/desktop/Airports_CL/data/raw/aircraft.csv", index=False)

 # Pobranie `Flight`
 Zapoznaj się z dokumentacją endpointu `flights`, następnie pobierz dane dotyczące ruchu lotniczego. Wyniki zapisz do ramki `flight_df`, a później do pliku `flight.csv`.

 Wskazówki:
 - Zwróć szczególną uwagę na konstrukcję endpointa,
 - Ze względu na wolumen danych, które tutaj się pobiorą, odradzamy zapisywanie danych bezpośrednio do ramki. Rekomendujemy podejście podobne do tego, z warsztatu na kursie `Python - analiza danych` - `Dzień 10 - Warsztat > Warsztat > Scrapowanie danych`,
 - Data początkowa danych to `2019-01-01`, zaś końcowa to `2020-03-31`, czyli 456 dni,
 - Ze względu na czas, jaki ten krok będzie się wykonywał, warto dodać w pętli instrukcję (lub kilka) `print`, aby monitorować przebieg wykonywania tego kroku.

 ## Sprawdzenie
 Uruchom kod poniżej, aby sprawdzić, czy ta część została poprawnie wykonana.

In [ ]:
flight_df_expected_shape = (9251880, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Zapis do pliku
 Tutaj zapisz ramkę `flight_df` do pliku `flight.csv` w katalogu `data/raw`.

 # Podsumowanie
 W tym notatniku wykonaliśmy podstawowy krok w analizie danych - pozyskaliśmy je. Są gotowe do dalszej pracy, czyli możemy załadować je na bazę danych, a następnie zapoznać się z tym, jakie informacje ze sobą niosą. Kolejne notatniki będą służyły właśnie tym celom.

In [ ]:
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego kroku."
print(msg)